In [167]:
import requests
import pandas as pd
import time

In [169]:
#from footballdata.org
API_TOKEN = "280e17f7e2524d5c8f85335cee4b38e1"


In [170]:
headers = {
    "X-Auth-Token": API_TOKEN
}

In [172]:
def get_table(competition_id, matchday):
        url = f'https://api.football-data.org/v4/competitions/{competition_id}/matches?matchday={matchday}&season=2023'
        response = requests.get(url, headers=headers)
        data = response.json()
        #wait 10 seconds so all data from the api can be collected
        time.sleep(1)

        rows = []  # collect rows here

        for match in  data['matches']:
            hometeam = match['homeTeam']
            awayteam = match['awayTeam']
            matchscore = match['score']['fullTime']
            referee = match['referees'][0]
            row = {
                'match_id' : match['id'],
                'match_date' : match['utcDate'],
                'match_status' : match['status'],
                'matchday' : match['matchday'],
                'stage' : match['stage'],
                'hometeam_id' :hometeam['id'],
                'hometeam' : hometeam['name'],
                'awayteam_id' : awayteam['id'],
                'awayteam' : awayteam['name'],
                'hometeam_score' : matchscore['home'],
                'awayteam_score' : matchscore['away'],
                'referee' : referee['name']
            }
            rows.append(row)
        return pd.DataFrame(rows)


In [173]:
def merge_matchdays(comp_id):
    dataframes = []
    matchdays = list(range(1,39))
    for match in matchdays:
        df = get_table(comp_id, match)
        time.sleep(5)
        dataframes.append(df) #collect all matchdays
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

In [174]:
Matches_Dataset = merge_matchdays(2021)

In [175]:
Matches_Dataset.shape

(380, 12)

In [177]:
Matches_Dataset.head(10)

,match_id,match_date,match_status,matchday,stage,hometeam_id,hometeam,awayteam_id,awayteam,hometeam_score,awayteam_score,referee
0,435943,2023-08-11T19:00:00Z,FINISHED,1,REGULAR_SEASON,328,Burnley FC,65,Manchester City FC,0,3,Craig Pawson
1,435944,2023-08-12T12:00:00Z,FINISHED,1,REGULAR_SEASON,57,Arsenal FC,351,Nottingham Forest FC,2,1,Michael Oliver
2,435945,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,1044,AFC Bournemouth,563,West Ham United FC,1,1,Peter Bankes
3,435946,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,397,Brighton & Hove Albion FC,389,Luton Town FC,4,1,David Coote
4,435947,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,62,Everton FC,63,Fulham FC,0,1,Stuart Attwell
5,435948,2023-08-12T14:00:00Z,FINISHED,1,REGULAR_SEASON,356,Sheffield United FC,354,Crystal Palace FC,0,1,John Brooks
6,435949,2023-08-12T16:30:00Z,FINISHED,1,REGULAR_SEASON,67,Newcastle United FC,58,Aston Villa FC,5,1,Andy Madley
7,435950,2023-08-13T13:00:00Z,FINISHED,1,REGULAR_SEASON,402,Brentford FC,73,Tottenham Hotspur FC,2,2,Robert Jones
8,435951,2023-08-13T15:30:00Z,FINISHED,1,REGULAR_SEASON,61,Chelsea FC,64,Liverpool FC,1,1,Anthony Taylor
9,435952,2023-08-14T19:00:00Z,FINISHED,1,REGULAR_SEASON,66,Manchester United FC,76,Wolverhampton Wanderers FC,1,0,Simon Hooper


In [95]:
#Matches_Dataset.to_csv('data/Matches_Dataset.csv', index=False)
from datetime import datetime

def save_dataframe(df, prefix, folder="../data"):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{folder}/{prefix}_{timestamp}.csv"
    df.to_csv(filename, index=False)
    print(f"Saved: {filename}")

In [203]:
save_dataframe(Matches_Dataset, 'matches_2023_24(raw)')

Saved: ../data/matches_2023_24(raw)_20250720_221550.csv


In [179]:
def get_teamdata():
    url = 'https://api.football-data.org/v4//competitions/2021/teams?season=2023'
    response = requests.get(url, headers=headers)
    data = response.json()
    time.sleep(1)

    rows = []
    for team in data['teams']:
        row ={
            'team_id' : team['id'],
            'team_name' : team['name'],
            'tla' : team['tla'],
            'year_founded' : team['founded'],
            'clubColors' : team['clubColors'],
            'stadium' : team['venue'],
            'crest' : team['crest'],
            }
        rows.append(row)

    return pd.DataFrame(rows)


In [180]:
team_df = get_teamdata()

In [181]:
team_df

,team_id,team_name,tla,year_founded,clubColors,stadium,crest
0,57,Arsenal FC,ARS,1886.0,Red / White,Emirates Stadium,https://crests.football-data.org/57.png
1,58,Aston Villa FC,AVL,1872.0,Claret / Sky Blue,Villa Park,https://crests.football-data.org/58.png
2,61,Chelsea FC,CHE,1905.0,Royal Blue / White,Stamford Bridge,https://crests.football-data.org/61.png
3,62,Everton FC,EVE,1878.0,Blue / White,Goodison Park,https://crests.football-data.org/62.png
4,63,Fulham FC,FUL,1879.0,White / Black,Craven Cottage,https://crests.football-data.org/63.png
5,64,Liverpool FC,LIV,1892.0,Red / White,Anfield,https://crests.football-data.org/64.png
6,65,Manchester City FC,MCI,1880.0,Sky Blue / White,Etihad Stadium,https://crests.football-data.org/65.png
7,66,Manchester United FC,MUN,1878.0,Red / White,Old Trafford,https://crests.football-data.org/66.png
8,67,Newcastle United FC,NEW,1881.0,Black / White,St. James' Park,https://crests.football-data.org/67.png
9,73,Tottenham Hotspur FC,TOT,1882.0,Navy Blue / White,Tottenham Hotspur Stadium,https://crests.football-data.org/73.png


In [202]:
save_dataframe(team_df, 'teams_2023_24(raw)')

Saved: ../data/teams_2023_24(raw)_20250720_221521.csv


In [160]:
#this was gotten from a different api
def get_scorers_data():
    API_KEY = '2bd1eaa7854b4d0862a6e0411ab93b89'
    headers = {'x-apisports-key': API_KEY}
    url = ' https://v3.football.api-sports.io/players/topscorers?league=39&season=2023'
    response = requests.get(url, headers=headers)
    data = response.json()
    time.sleep(1)
    
    rows = []
    for player in  data['response']:
        stats =  player['statistics'][0]
        row = {
                'player_id' : player['player']['id'],
                'player_name' : player['player']['name'],
                'nationality' : player['player']['nationality'],
                'position' : stats['games']['position'],
                'team_id' : stats['team']['id'],
                'team_name' : stats['team']['name'],
                'appearences': stats['games']['appearences'],
                'minutes': stats['games']['minutes'],
                'goals' : stats['goals']['total'],
                'assists' : stats['goals']['assists'],
                'penalty_goals' : stats['penalty']['scored'],
            }

        rows.append(row)
    return pd.DataFrame(rows)        

In [161]:
scorers_df = get_scorers_data()

In [163]:
scorers_df = scorers_df.drop('team_id', axis=1)

In [183]:
#change the team Id to the v4 version from the first api
scorers_df['team_id'] = [65,61,67,58,65,1044,64,73,563,354,57,61,351,57,402,57,76,76,64,73]

In [165]:
scorers_df

,player_id,player_name,nationality,position,team_name,appearences,minutes,goals,assists,penalty_goals,team_id
0,1100,E. Haaland,Norway,Attacker,Manchester City,31,2559,27,5,7,65
1,152982,C. Palmer,England,Midfielder,Chelsea,33,2618,22,11,9,61
2,2864,A. Isak,Sweden,Attacker,Newcastle,30,2267,21,2,5,67
3,19366,O. Watkins,England,Attacker,Aston Villa,37,3227,19,13,0,58
4,631,P. Foden,England,Midfielder,Manchester City,35,2868,19,8,0,65
5,18883,D. Solanke,England,Attacker,Bournemouth,38,3332,19,3,2,1044
6,306,Mohamed Salah,Egypt,Attacker,Liverpool,32,2536,18,10,5,64
7,186,Son Heung-Min,Korea Republic,Attacker,Tottenham,35,2948,17,10,2,73
8,19428,J. Bowen,England,Attacker,West Ham,34,3021,16,6,0,563
9,25927,J. Mateta,France,Attacker,Crystal Palace,35,2284,16,5,2,354


In [204]:
save_dataframe(scorers_df, 'scorers_2023_24(raw)')

Saved: ../data/scorers_2023_24(raw)_20250720_221604.csv


In [198]:
def get_standings():
    url = 'https://api.football-data.org/v4//competitions/2021/standings?season=2023'
    response = requests.get(url, headers=headers)
    data = response.json()
    time.sleep(1)
    
    rows = []
    
    for team in data['standings'][0]['table']:
        row = {
            'team_id' : team['team']['id'],
            'team_name' : team['team']['name'],
            'position' : team['position'],
            'games_played' : team['playedGames'],
            'form': team['form'],
            'wins': team['won'],
            'draws': team['draw'],
            'losses': team['lost'],
            'points': team['points'],
            'goalsFor': team['goalsFor'],
            'goals_against': team['goalsAgainst'],
            'goal_difference': team['goalDifference']
        }

        rows.append(row)
        
    return pd.DataFrame(rows)

In [199]:
standings = get_standings() 

In [200]:
standings

,team_id,team_name,position,games_played,form,wins,draws,losses,points,goalsFor,goals_against,goal_difference
0,65,Manchester City FC,1,38,"W,W,W,W,W",28,7,3,91,96,34,62
1,57,Arsenal FC,2,38,"W,W,W,W,W",28,5,5,89,91,29,62
2,64,Liverpool FC,3,38,"W,D,W,D,L",24,10,4,82,86,41,45
3,58,Aston Villa FC,4,38,"L,D,L,D,W",20,8,10,68,76,61,15
4,73,Tottenham Hotspur FC,5,38,"W,L,W,L,L",20,6,12,66,74,61,13
5,61,Chelsea FC,6,38,"W,W,W,W,W",18,9,11,63,77,63,14
6,67,Newcastle United FC,7,38,"W,L,D,W,W",18,6,14,60,85,62,23
7,66,Manchester United FC,8,38,"W,W,L,L,D",18,6,14,60,57,58,-1
8,563,West Ham United FC,9,38,"L,W,L,D,L",14,10,14,52,60,74,-14
9,354,Crystal Palace FC,10,38,"W,W,W,D,W",13,10,15,49,57,58,-1


In [205]:
save_dataframe(standings, 'standings_2023_24(raw)')

Saved: ../data/standings_2023_24(raw)_20250720_221620.csv


In [206]:
df = pd.read_csv('../data/matches_2023_24(raw)_20250720_221550.csv')

In [208]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 380 entries, 0 to 379
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   match_id        380 non-null    int64 
 1   match_date      380 non-null    object
 2   match_status    380 non-null    object
 3   matchday        380 non-null    int64 
 4   stage           380 non-null    object
 5   hometeam_id     380 non-null    int64 
 6   hometeam        380 non-null    object
 7   awayteam_id     380 non-null    int64 
 8   awayteam        380 non-null    object
 9   hometeam_score  380 non-null    int64 
 10  awayteam_score  380 non-null    int64 
 11  referee         380 non-null    object
dtypes: int64(6), object(6)
memory usage: 35.8+ KB


In [209]:
df['match_date'] = pd.to_datetime(df['match_date'])

In [210]:
df.drop(columns=['hometeam_id', 'awayteam_id'], inplace =True)

In [211]:
df = df.rename(columns={
    'id': 'match_id',
    'match_date': 'date',
    'hometeam': 'home_team',
    'awayteam': 'away_team',
    'hometeam_score': 'home_goals',
    'awayteam_score': 'away_goals'
})

In [212]:
def get_result(row):
    if row['home_goals'] > row['away_goals']:
        return 'H'
    elif row['home_goals'] < row['away_goals']:
        return 'A'
    else:
        return 'D'

df['result'] = df.apply(get_result, axis=1)

In [213]:
df.head()

,match_id,date,match_status,matchday,stage,home_team,away_team,home_goals,away_goals,referee,result
0,435943,2023-08-11 19:00:00+00:00,FINISHED,1,REGULAR_SEASON,Burnley FC,Manchester City FC,0,3,Craig Pawson,A
1,435944,2023-08-12 12:00:00+00:00,FINISHED,1,REGULAR_SEASON,Arsenal FC,Nottingham Forest FC,2,1,Michael Oliver,H
2,435945,2023-08-12 14:00:00+00:00,FINISHED,1,REGULAR_SEASON,AFC Bournemouth,West Ham United FC,1,1,Peter Bankes,D
3,435946,2023-08-12 14:00:00+00:00,FINISHED,1,REGULAR_SEASON,Brighton & Hove Albion FC,Luton Town FC,4,1,David Coote,H
4,435947,2023-08-12 14:00:00+00:00,FINISHED,1,REGULAR_SEASON,Everton FC,Fulham FC,0,1,Stuart Attwell,A


In [215]:
df.drop('referee', axis=1, inplace=True)

In [220]:
save_dataframe(df, 'matches_clean')

Saved: ../data/matches_clean_20250720_223207.csv
